# <center> Fonctions projet chef-d'oeuvre </center>

In [34]:
import pandas as pd
import csv
from tqdm.notebook import tqdm_notebook
import isbnlib
from isbnlib import canonical, meta

---

## <center> Web scraping </center>

## Créer un nouveau csv

In [35]:
def creation_csv(nom): # il suffit d'indiquer le nom du csv (sans l'extension)
    header = ['Auteur', 'Titre', 'Edition', 'Date', 'ISBN']
    with open(nom +'.csv', 'w') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(header)
    print("fichier créé avec le nom suivant :", nom)

## Web-scraping 

In [37]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm.notebook import tqdm_notebook


def web_scraping(nom_csv, nb_livre, nombre, site):
    liste_totale = []
    for livre in tqdm_notebook(range(nb_livres)) : 
        erreur = 0
        
        try : 
            url = site + str(nombre)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')

            liste = []

            # auteur 
            auteur = soup.find("h2")
            auteur = auteur.text.strip('\n')
            liste.append(auteur)

            # titre
            titre = soup.find("h1")
            titre = titre.text.strip('\n')
            liste.append(titre)

            # edition
            edition = soup.find("h3")
            edition = edition.text.strip('\n')
            liste.append(edition)

            # date
            date = soup.findAll("dd")
            # on récupère la date dans une variable temporaire
            var_temp = date[5].text
            # on split la date et on ne récupère que l'année
            split = var_temp.split("/")
            date = split[-1]
            # # on ajoute la date (format : int) à la liste
            # liste.append(int(date))
            liste.append(date)

            # ISBN
            # Récupérer l'isbn
            info = soup.findAll("dd")
            # on récupère l'isbn dans une variable temporaire
            isbn = info[2].text
            liste.append(isbn)

            liste_totale.append(liste)

            nombre += 1

        except :

            nombre += 1
            erreur += 1


    print("Nb d'erreurs :", erreur)

    # insérer les informations pour chaque livre (à partir de la liste totale)
    with open(nom_csv+'.csv', 'a', encoding='UTF8') as f:
            for x in tqdm_notebook(liste_totale) :
                writer = csv.writer(f)
                writer.writerow(x)

            # message de réussite
            print("Insertion des données réussies.")

---

## <center> Formatage des données </center>

## Ouvrir un df

In [38]:
import pandas as pd

def open_df(file_name):
    df = pd.read_csv(file_name + '.csv')
    return df

## Récupérer des infos sur le df

In [40]:
def info_df (file_name):
    df = open_df(file_name)
    df.info()

## Supprimer doublons dans le csv

In [42]:
def delete_duplicates(file_name):
    # Suppression
    df.drop_duplicates(subset=None, inplace=True)
    # Sauvegarde = écrase le csv
    df.to_csv(file_name + ".csv", index=False)
    print("Les doublons ont bien été supprimés.")
    return df

## Formater la colonne 'Titre'

In [43]:
# def formater_titre(file_name):
#     df['Titre'] = df['Titre'].replace(to_replace =["\n", "\r"], 
#                                 value ="")

#     # Sauvegarde du csv 
#     df.to_csv(file_name + ".csv", index=False)
#     print("Formatage de la colonne 'Titre' : OK")
#     return df

## Formater la colonne 'Auteur'

In [44]:
def formater_auteur(file_name):
    # replace
    df['Auteur'] = df['Auteur'].str.replace("De ", "", regex=False)
#     # Sauvegarde du csv 
#     df.to_csv(file_name + ".csv", index=False)
    # # None 
    df['Auteur'] = df['Auteur'].replace(to_replace =["[none]", "[None]", "nones", "none", "Inconnu", "inconnu", "Xxx", " . ",
                                                          "nouvelles", "nouvelle", "roman", "romans", "poème", "poèmes","Poèmes", "Poème", "poésie", "poésies", "récit", 
                                                          "récits", "Livre", "Livres", "livre", "livres", "Volume 2", "et autres nouvelles",
                                                          "<<<<< ", "<<<<<", "Volume 1", "[nouvelles]", "livre du professeur", "l'intégrale",
                                                          "Pro", "Prof", "théâtre", "Théâtre", "A Preciser", "Po", "France", "- Roman", "Roman",
                                                          "3","2", "Anonyme", "anthologie", "Inconnu / Inconnu", "essai", "essais", "5e", "Tome 1"], 
                                value ="None")

    # Collectif
    df['Auteur'] = df['Auteur'].replace(to_replace =["De Collectif", "Collectifs", "collectif", " Collectif", "De Collectifs",
                                                          "Collectif d'auteurs", " Collectif d'auteurs", "Collectifs Jeunesse", "Collectifs Lois",
                                                          "Collectif,", "Collectifs Gallimard"],
                                value = "Collectif")
    # Sauvegarde du csv 
    df.to_csv(file_name + ".csv", index=False)
    print("Formatage de la colonne 'Auteur' : OK")
    return df

## Formatage de la colonne 'Edition'

In [45]:
def formater_edition(file_name):
    # Formatage des éditions 
    df['Edition'] = df['Edition'].str.replace("L'Harmattan", "Harmattan", regex=False)
    df['Edition'] = df['Edition'].str.replace("Presses universitaires de France", "PUF", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions Ellipses", "Ellipses", regex=False)
    df['Edition'] = df['Edition'].str.replace("Éditions du Rocher", "Rocher", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions du Rocher", "Rocher", regex=False)
    df['Edition'] = df['Edition'].str.replace("Editions ", "", regex=False)
    df['Edition'] = df['Edition'].str.replace("Éditions ", "", regex=False)

    # Sauvegarde du csv 
    df.to_csv(file_name + ".csv", index=False)
    print("Formatage de la colonne 'Edition' : OK")
    return df

---

## Traitement des ISBN

In [46]:
# Suppression des ISBN non chiffrés
from tqdm.notebook import tqdm_notebook

def traitement_isbn(nom_csv):
    liste=[]
    index = 0
    for x in tqdm_notebook(df['ISBN']):
        try :
            test = int(x)
            index += 1
        except :
            df.drop([index], axis=0, inplace=True)
            index +=1 

    # Sauvegarde
    df.to_csv(nom_csv+".csv", index=False)
    
    return df


In [47]:
# Trouver des ISBN aberrants :

def min_max_isbn():
    print("Min : ", df['ISBN'].min())
    print("Max : ", df['ISBN'].max())

In [48]:
# Supprimer les ISBN aberrants


def delete_bad_isbn(csv_name):
    index = -1
    liste = []

    for x in df['ISBN']:
        index += 1
        if x <= 9780000000000 :
            liste.append(index)
        
    # message    
    print("Nb d'erreurs d'ISBN :", len(liste))
    # delete
    df.drop(liste, axis=0, inplace=True)
    # save
    df.to_csv(csv_name + ".csv", index=False)
    
    return df.info()

## Traitement sur les dates

In [49]:
# Formatage
def formater_date(csv_name):
    # formatage des dates
    df['Date'] = df['Date'].replace(to_replace =["©","© ","Imp ","imp","imp ","impr ","Impr ","DL ","dl ","ilmpr","ilmpr ",
        "cop","Cop","ca","(c)","Paris PUF ","Paris PUF","Paris Gallimard ","Paris Gallimard","Paris Flammarion ",
        "Paris Flammarion","Paris Les Belles Lettres ","Paris Les Belles Lettres","Paris L'Harmattan ","Paris L'Harmattan",
        "Paris Hachette ","Paris Hachette","Paris Colin","Paris Colin " 
        ],
        value = "")
    # suppression des dates non numériques
    liste=[]
    index = 0
    for x in tqdm_notebook(df['Date']):
        try :
            test = int(x)
            index += 1
        except :
            df.drop([index], axis=0, inplace=True)
            index +=1 

    # Sauvegarde du csv 
    df.to_csv(csv_name +".csv", index=False)
    
    return df

In [50]:
# Statistiques sur les dates

def stats_date():
    print("Moyenne :", round(df.Date.mean(), 2))
    print("Min :", df.Date.min())
    print("Max :", df.Date.max())

In [51]:
# supression des dates aberrantes 

def delete_bad_dates(csv_name):
    nombre = 0
    index = -1
    for x in df['Date']:
        index += 1
        if x < 1700 :
            df.drop([index], axis=0, inplace=True)
            nombre += 1
    # Save
    df.to_csv(csv_name+".csv", index=False)

## Remplacer "." par "None"

In [52]:
def remplacer_point(file_name):
    df = open_df(file_name)
    # filtre sélectionnant tous les "."
    filtre_point = df[df['Auteur']== "."]

    # création d'une liste pour les index
    liste_index = []
    for index, row in filtre_point.iterrows(): 
        liste_index.append(index)   
    print("Nombre d'index dans la liste :", len(liste_index))

    # Remplacement par None
    for x in tqdm_notebook(liste_index) :
        df.loc[x, 'Auteur']= "None"
        df.to_csv("test.csv", index=False)
        
    # Save
    df.to_csv(file_name+".csv", index=False)

## Remplacer les "None" par les noms des auteurs

In [53]:
def traiter_None(file_name):
    
    erreur = 0
    
    df = open_df(file_name)
    # Filtre avec les None
    filtre_None = df[df['Auteur'] == 'None']
    # Récupérer les index du filtre_None
    liste_index = filtre_None.index

    for index in tqdm_notebook(liste_index):
        # Récupérer l'ISBN
        ISBN = df.loc[index, 'ISBN']
        # recherche info : on récupère l'auteur
        try :
            isbn = canonical(str(ISBN))
            data = isbnlib.meta(isbn, service='goob') # changer le service
            try :
                auteur = data['Authors']

                ISBN = df.loc[index, 'ISBN']
                # recherche info : on récupère l'auteur
                isbn = canonical(str(ISBN))
                data = isbnlib.meta(isbn, service='goob') # changer le service
                try :
                    auteur = data['Authors']

                    if len(auteur) == 1 :
                        for x in auteur :
                            auteur = x
                    else : 
                        continue

                except :
                    erreur +=1
                    continue
            except :
                erreur += 1
                continue
        except :
            erreur += 1
            continue

        # On change la valeur
        df.loc[index, 'Auteur']= auteur

    # Save
    df.to_csv(file_name+".csv", index=False)
    
    print("Nombre d'erreurs : ", erreur)

## Concaténation

In [54]:
def concat_csv(csv_1, csv_2, nom_csv):
    df = pd.concat(
    map(pd.read_csv, [csv_1+'.csv',csv_2+'.csv']), ignore_index=True)
    # Sauvegarde
    df.to_csv(nom_csv+".csv", index=False)

---

# <center> Traitement du csv </center>

In [72]:
nom_csv = 'bdd_1'

In [83]:
df = open_df(nom_csv)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39658 entries, 0 to 39657
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Auteur   39536 non-null  object
 1   Titre    39658 non-null  object
 2   Edition  39658 non-null  object
 3   Date     39658 non-null  int64 
 4   ISBN     39658 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


## Formater Auteur/Edition

In [74]:
def traitement_csv(file_name):
    df = open_df(file_name)
    df = delete_duplicates(file_name)
    df = formater_auteur(file_name)
    df = formater_edition(file_name)
    df = remplacer_point(file_name)
    return df

In [75]:
traitement_csv(nom_csv)

Les doublons ont bien été supprimés.
Formatage de la colonne 'Auteur' : OK
Formatage de la colonne 'Edition' : OK
Nombre d'index dans la liste : 327


  0%|          | 0/327 [00:00<?, ?it/s]

## Formater ISBN/Dates

In [78]:
traitement_isbn(nom_csv)
df = open_df(nom_csv)

delete_bad_isbn(nom_csv)
df = open_df(nom_csv)

formater_date(nom_csv)
df = open_df(nom_csv)

delete_bad_dates(nom_csv)
df = open_df(nom_csv)
df.info()

  0%|          | 0/60133 [00:00<?, ?it/s]

Nb d'erreurs d'ISBN : 2049
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39690 entries, 0 to 41738
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Auteur   39682 non-null  object
 1   Titre    39690 non-null  object
 2   Edition  39690 non-null  object
 3   Date     39690 non-null  object
 4   ISBN     39690 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


  0%|          | 0/39690 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39658 entries, 0 to 39657
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Auteur   39650 non-null  object
 1   Titre    39658 non-null  object
 2   Edition  39658 non-null  object
 3   Date     39658 non-null  int64 
 4   ISBN     39658 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


In [84]:
df['Auteur'].value_counts().head(10)

Collectif                                          1279
None                                                815
auzias/al. dominique                                108
Collectif,                                           34
Dominique Auzias, Jean-Paul Labourdette              28
N.C.                                                 26
Adaptation Piccolia                                  25
Agatha Christie                                      21
Barbara Cartland                                     21
Manufacture française des pneumatiques Michelin      19
Name: Auteur, dtype: int64

## Remplacer les '.' par "None"

In [80]:
remplacer_point(nom_csv)

Nombre d'index dans la liste : 0


0it [00:00, ?it/s]

## Remplacer les None

In [81]:
traiter_None(nom_csv)

  0%|          | 0/3259 [00:00<?, ?it/s]

Nombre d'erreurs :  491


## Concaténation

In [85]:
concat_csv("54", nom_csv, "55")

---